In [ ]:
import glob
import os
import pathlib
import re

import pandas as pd

In [ ]:
try:
    from google.colab import drive
    drive.mount('/gdrive')
    core_path = "/path name/"
except ModuleNotFoundError:
    core_path = ""

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
spectra_filepaths = sorted(glob.glob(os.path.join(core_path, "data/lvl0/", "smass2/*spfit*")))

In [ ]:
des_file_paths = spectra_filepaths[:-8]
non_file_paths = spectra_filepaths [-8:]

des_file_paths_df = pd.DataFrame(des_file_paths, columns=["FilePath"])
non_file_paths_df = pd.DataFrame(des_file_paths, columns=["FilePath"])

In [ ]:
import re

des_file_paths_df.loc[:, "DesNr"] = des_file_paths_df["FilePath"].apply(
    lambda x: int(re.search(r"smass2/a(.*).spfit", x).group(1))
)

non_file_paths_df.loc[:, "DesNr"] = non_file_paths_df["FilePath"].apply(
    lambda x: re.search(r"smass2/a(.*).spfit", x).group(1)
)


In [ ]:
asteroid_class_df = pd.read_csv(os.path.join(core_path, "data/lvl0/", "Bus.Taxonomy.txt"),
                               skiprows=21,
                               sep="\t",
                               names=["Name", "Tholen_Class", "Bus_Class", "unknown1", "unknown2"])

In [ ]:
asteroid_class_df

,Name,Tholen_Class,Bus_Class,unknown1,unknown2
0,1 Ceres,G,C,NaN,NaN
1,2 Pallas,B,B,NaN,NaN
2,3 Juno,S,Sk,NaN,NaN
3,4 Vesta,V,V,NaN,NaN
4,5 Astraea,S,S,NaN,NaN
...,...,...,...,...,...
1442,1998 WS,NaN,Sr,NaN,NaN
1443,1998 WZ6,NaN,V,NaN,NaN
1444,1999 EE5,NaN,S,NaN,NaN
1445,1999 FA,NaN,S,NaN,NaN


In [ ]:
asteroid_class_df.loc[:, "Name"] = asteroid_class_df["Name"].apply(lambda x: x.strip()).copy()

des_ast_class_df = asteroid_class_df[:1403].copy()
non_ast_class_df = asteroid_class_df[:1403].copy()

In [ ]:
des_ast_class_df.loc[:, "DesNr"] = des_ast_class_df["Name"].apply(lambda x: int(x.split(" ")[0]))

des_ast_class_join_df = des_ast_class_df.merge(des_file_paths_df, on="DesNr")

In [ ]:
non_ast_class_df.loc[:, "DesNr"] = non_ast_class_df["Name"].apply(lambda x: x.replace(" ", ""))

non_ast_class_join_df = non_ast_class_df.merge(non_file_paths_df, on="DesNr")

In [ ]:
import pandas as pd
asteroids_df = pd.concat([des_ast_class_join_df, non_ast_class_join_df], axis=0)
asteroids_df.reset_index(drop=True, inplace=True)
asteroids_df.drop(columns=["Tholen_Class", "unknown1", "unknown2"], inplace=True)
asteroids_df.dropna(subset=["Bus_Class"], inplace=True)


In [ ]:
asteroids_df


,Name,Bus_Class,DesNr,FilePath
0,1 Ceres,C,1,/gdrive/MyDrive/Space-ML/asteroid_taxonomy/dat...
1,2 Pallas,B,2,/gdrive/MyDrive/Space-ML/asteroid_taxonomy/dat...
2,3 Juno,Sk,3,/gdrive/MyDrive/Space-ML/asteroid_taxonomy/dat...
3,4 Vesta,V,4,/gdrive/MyDrive/Space-ML/asteroid_taxonomy/dat...
4,5 Astraea,S,5,/gdrive/MyDrive/Space-ML/asteroid_taxonomy/dat...
...,...,...,...,...
1328,11785 1973 AW3,Xc,11785,/gdrive/MyDrive/Space-ML/asteroid_taxonomy/dat...
1329,11906 1992 AE1,Sq,11906,/gdrive/MyDrive/Space-ML/asteroid_taxonomy/dat...
1330,12281 1990 WA5,X,12281,/gdrive/MyDrive/Space-ML/asteroid_taxonomy/dat...
1331,17480 1991 PE10,S,17480,/gdrive/MyDrive/Space-ML/asteroid_taxonomy/dat...


In [ ]:
asteroids_df.loc[:, "SpectrumDF"] = \
    asteroids_df["FilePath"].apply(lambda x: pd.read_csv(x, sep="\t",
                                                         names=["Wavelength_in_micron",
                                                                "Reflectance_norm550nm"]
                                                        )
                                  )
# Reset the index
asteroids_df.reset_index(drop=True, inplace=True)

# Convert the Designation Number to string
asteroids_df.loc[:, "DesNr"] = asteroids_df["DesNr"].astype(str)


NameError: ignored

In [ ]:
pathlib.Path(os.path.join(core_path, "data/lvl1")).mkdir(parents=True, exist_ok=True)

asteroids_df.to_pickle(os.path.join(core_path, "data/lvl1/", "asteroids_merged.pkl"), protocol=4)